In [1]:
! pip install gymnasium-robotics

  Using cached gymnasium_robotics-1.2.4-py3-none-any.whl.metadata (8.1 kB)
  Using cached mujoco-2.3.7.tar.gz (593 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
  Using cached pettingzoo-1.24.3-py3-none-any.whl.metadata (8.5 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached imageio-2.35.1-py3-none-any.whl.metadata (4.9 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached glfw-2.7.0-py2.py27.py3.py30.py31.py32.py33.py34.py35.py36.py37.py38-none-win_amd64.whl.metadata (5.4 kB)
  Using cached PyOpenGL-3.1.7-py3-none

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [72 lines of output]
      C:\Users\cesar\miniconda3\envs\metarl\Lib\site-packages\setuptools\_distutils\dist.py:268: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      C:\Users\cesar\miniconda3\envs\metarl\Lib\site-packages\setuptools\_distutils\dist.py:268: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\bindings_test.py -> build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\gl_context.py -> build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\renderer.py -> build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\renderer_test.py -> build\lib.win-amd64-cpython-312\

In [2]:
! pip install mujoco==2.3.0

  Using cached mujoco-2.3.0.tar.gz (144 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached glfw-2.7.0-py2.py27.py3.py30.py31.py32.py33.py34.py35.py36.py37.py38-none-win_amd64.whl.metadata (5.4 kB)
  Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached PyOpenGL-3.1.7-py3-none-any.whl.metadata (3.2 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached glfw-2.7.0-py2.py27.py3.py30.py31.py32.py33.py34.py35.py36.py37.py38-none-win_amd64.whl (493 kB)
Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached PyOpenGL-3.1.7-py3-none-any.whl (2.4 MB)
  Running setup.py clean for mujoco
Failed to build mujoco


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      C:\Users\cesar\miniconda3\envs\metarl\Lib\site-packages\setuptools\_distutils\dist.py:268: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      C:\Users\cesar\miniconda3\envs\metarl\Lib\site-packages\setuptools\_distutils\dist.py:268: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\bindings_test.py -> build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\render_test.py -> build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\rollout.py -> build\lib.win-amd64-cpython-312\mujoco
      copying mujoco\rollout_test.py -> build\lib.win-amd64-cpython-312\m

In [6]:
import gymnasium as gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import json
import sys
import os

# Add the parent directory to sys.path if __file__ is defined
#if '__file__' in globals():
#sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize  # , SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

from stable_baselines3.common.utils import set_random_seed

from train_test import train_model, test_model
from dr_wrapper import DR_Wrapper, Auto_DR_Wrapper

# Set to True to train or test the model
TRAIN = True
TEST = False

# Load settings from JSON file
with open('hopper_settings.json', 'r') as f:
    settings = json.load(f)

# Extract settings
env_name = settings["env_name"]
train_steps = settings["train_steps"]
n_envs = settings["n_envs"]
n_steps = settings["n_steps"]
batch_size = settings["batch_size"]
n_epochs = settings["n_epochs"]
gamma = settings["gamma"]
ent_coef = settings["ent_coef"]
gae_lambda = settings["gae_lambda"]
sde_sample_freq = settings["sde_sample_freq"]
activation_fn = getattr(nn, settings["activation_fn"])
policy_kwargs = settings["policy_kwargs"]
policy_kwargs["activation_fn"] = getattr(nn, policy_kwargs["activation_fn"])
dr_specs = settings["dr_specs"]

if __name__ == "__main__":
    if TRAIN:
        #for seed in [42, 142, 242, 342, 442]:
        for seed in [42]:
            model_path, env_path = train_model(
                env_name, seed, train_steps, policy_kwargs, dr_specs, n_envs, n_steps, batch_size, n_epochs, gamma, ent_coef, gae_lambda, sde_sample_freq
            )
            print(f'Model saved at {model_path}')
            print(f'Env saved at {env_path}')

    if TEST:
        for seed in [42, 142, 242, 342, 442]:
            model_path = f'{env_name}_{seed}_logs/{env_name}_{seed}.ckpt'
            env_path = f'{env_name}_{seed}_logs/{env_name}_vecnormalize_{seed}.pkl'
            perf_path = test_model(model_path, env_path, env_name, seed, n_envs, dr_specs=dr_specs)
            print(f'Performance saved at {perf_path}')

Using SDE for Hopper-v4
Using CUDA
Folder './Hopper-v4_42_logs/' created successfully.
Using cuda:0 device


c:\Users\cesar\miniconda3\envs\R-ML\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.model to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.model` for environment variables or `env.get_wrapper_attr('model')` that will search the reminding wrappers.
  logger.warn(


ImportError: Trying to log data to tensorboard but tensorboard is not installed.